In [1]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string

import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize

from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df_dialogues = None

for i in range(1,6):
    
    file_dir = f"D:\ATLAS\Sem 6\Minor project\processed_text_files\Session_{i}_dialogue.csv"

    temp_df = pd.read_csv(file_dir, index_col=0)
    df_dialogues = pd.concat([df_dialogues, temp_df])

In [3]:
df_emotions = None

for i in range(1,6):
    
    file_dir = f"D:\ATLAS\Sem 6\Minor project\processed_text_files\Session_{i}_emotion.csv"

    temp_df = pd.read_csv(file_dir, index_col=0)
    df_emotions = pd.concat([df_emotions, temp_df])

In [4]:
df_dialogues.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10239 entries, 0 to 2196
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   file     10239 non-null  object
 1   speaker  10239 non-null  object
 2   dialog   10172 non-null  object
dtypes: object(3)
memory usage: 320.0+ KB


In [5]:
df_emotions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10039 entries, 0 to 2169
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   file     10039 non-null  object
 1   speaker  10039 non-null  object
 2   emotion  10039 non-null  object
dtypes: object(3)
memory usage: 313.7+ KB


# Feature Engineering

In [6]:
df_emotions = df_emotions[(df_emotions['emotion'] != "xxx") & (df_emotions['emotion'] != "fru")]


In [7]:
df_emotions["emotion"].value_counts()

emotion
neu    1708
ang    1103
sad    1084
exc    1041
hap     595
sur     107
fea      40
oth       3
dis       2
Name: count, dtype: int64

In [8]:
df_combined = pd.merge(df_dialogues, df_emotions, on="speaker")

In [9]:
df_combined[df_combined['emotion'] == 'neu'][:5]

,file_x,speaker,dialog,file_y,emotion
0,Ses01F_impro01,Ses01F_impro01_F000,Excuse me.,Ses01F_impro01,neu
1,Ses01F_impro01,Ses01F_impro01_F001,Yeah.,Ses01F_impro01,neu
2,Ses01F_impro01,Ses01F_impro01_F002,Is there a problem?,Ses01F_impro01,neu
3,Ses01F_impro01,Ses01F_impro01_F005,Well what's the problem? Let me change it.,Ses01F_impro01,neu
6,Ses01F_impro01,Ses01F_impro01_F014,"Clearly. You know, do you have like a supervis...",Ses01F_impro01,neu


In [10]:
df_combined[df_combined['emotion'] == 'hap'][:5]

,file_x,speaker,dialog,file_y,emotion
36,Ses01F_impro03,Ses01F_impro03_F000,Well Vegas was awesome.,Ses01F_impro03,hap
38,Ses01F_impro03,Ses01F_impro03_F001,"And, um, I got married.",Ses01F_impro03,hap
40,Ses01F_impro03,Ses01F_impro03_F002,Yeah. In the old town part.,Ses01F_impro03,hap
42,Ses01F_impro03,Ses01F_impro03_F004,Um- Yes. It was very romantic.,Ses01F_impro03,hap
43,Ses01F_impro03,Ses01F_impro03_F005,It was at the slot machines.,Ses01F_impro03,hap


In [11]:
df_combined[df_combined['emotion'] == 'ang'][:5]

,file_x,speaker,dialog,file_y,emotion
4,Ses01F_impro01,Ses01F_impro01_F012,That's out of control.,Ses01F_impro01,ang
5,Ses01F_impro01,Ses01F_impro01_M011,I don't understand why this is so complicated ...,Ses01F_impro01,ang
7,Ses01F_impro01,Ses01F_impro01_M013,Yeah. Do you want to see my supervisor? Huh? Y...,Ses01F_impro01,ang
114,Ses01F_impro04,Ses01F_impro04_F028,I'm not frustrated.,Ses01F_impro04,ang
115,Ses01F_impro04,Ses01F_impro04_F029,Who do you think you are?,Ses01F_impro04,ang


In [12]:
df_combined[df_combined['emotion'] == 'sad'][:5]

,file_x,speaker,dialog,file_y,emotion
8,Ses01F_impro02,Ses01F_impro02_F000,Did you get the mail? So you saw my letter?,Ses01F_impro02,sad
9,Ses01F_impro02,Ses01F_impro02_F001,Yeah. I know.,Ses01F_impro02,sad
10,Ses01F_impro02,Ses01F_impro02_M001,I don't understand. You've already done so muc...,Ses01F_impro02,sad
11,Ses01F_impro02,Ses01F_impro02_F002,I don't know. I put in that. request too. They...,Ses01F_impro02,sad
13,Ses01F_impro02,Ses01F_impro02_F004,"There's people that have given more though, yo...",Ses01F_impro02,sad


In [13]:
df_combined[df_combined['emotion'] == 'sur'][:5]

,file_x,speaker,dialog,file_y,emotion
39,Ses01F_impro03,Ses01F_impro03_M002,Shut up. No- in Vegas?,Ses01F_impro03,sur
62,Ses01F_impro03,Ses01F_impro03_M017,Really? Penny slots?,Ses01F_impro03,sur
64,Ses01F_impro03,Ses01F_impro03_M018,Wow.,Ses01F_impro03,sur
169,Ses01F_impro05,Ses01F_impro05_M032,"Badge number? Uh, no. You can just take my emp...",Ses01F_impro05,sur
171,Ses01F_impro05,Ses01F_impro05_M033,Why? Have I done something wrong? Have I given...,Ses01F_impro05,sur


In [14]:
df_combined['emotion'] = df_combined['emotion'].replace({'exc': 'hap', 'fru': 'ang', 'oth': 'sur', 'dis': 'ang', 'fea': 'neu'})

In [15]:
df_combined['emotion'].value_counts()

emotion
neu    1748
hap    1636
ang    1105
sad    1084
sur     110
Name: count, dtype: int64

In [16]:
first_idx = [0,36,4,8,39]

In [17]:
dialog_len = []

for idx, row in df_combined.iterrows():
    length = len(row.iloc[2])
    dialog_len.append(length)

df_combined["dialog_len"] = dialog_len

In [18]:
df_combined[["dialog_len", "emotion"]].describe(include="all")

,dialog_len,emotion
count,5683.000000,5683
unique,NaN,5
top,NaN,neu
freq,NaN,1748
mean,57.476333,NaN
std,51.648740,NaN
min,3.000000,NaN
25%,21.000000,NaN
50%,42.000000,NaN
75%,78.000000,NaN


In [19]:
df_combined[["emotion", "dialog_len"]].groupby("emotion").mean()

,dialog_len
emotion,
ang,63.523982
hap,61.575795
neu,50.928490
sad,58.262915
sur,32.054545


In [20]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun


In [21]:
pos_tag(["running", "runner"])

[('running', 'VBG'), ('runner', 'NN')]

In [22]:
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    
    tokens = word_tokenize(sentence)
    tagged_tokens = pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]
    
    return ' '.join(lemmatized)


In [23]:
df_combined['dialog'].iloc[10]

"I don't understand. You've already done so much, I don't know why do you have to go back."

In [24]:
df_combined['dialog'] = df_combined['dialog'].map(lemmatize_sentence)

In [25]:
df_combined['dialog'].iloc[10]

"I do n't understand . You 've already do so much , I do n't know why do you have to go back ."

# Data Processing

In [26]:
x = df_combined["dialog"].values
y = df_combined["emotion"].values

In [27]:
len(max(x))

20

In [28]:
for idx in first_idx:
    print(y[idx])

neu
hap
ang
sad
sur


In [29]:
le = LabelEncoder()
y = le.fit_transform(y)

In [30]:
for idx in first_idx:
    print(y[idx])

2
1
0
3
4


In [31]:
labels = {0: 'ang', 1: 'hap', 2: 'neu', 3: 'sad', 4: 'sur'}

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42, stratify=y)

print("Length of training set: ", len(x_train))
print("Length of test set: ", len(x_test))

Length of training set:  5114
Length of test set:  569


In [33]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

print("Length of training set: ", len(x_train))
print("Length of validation set: ", len(x_val))

Length of training set:  4602
Length of validation set:  512


In [34]:
# Prepare the dataset
x_train = tf.constant(x_train)
x_val = tf.constant(x_val)
x_test = tf.constant(x_test)


y_train = keras.utils.to_categorical(y_train, num_classes=5)
y_val = keras.utils.to_categorical(y_val, num_classes=5)
y_test = keras.utils.to_categorical(y_test, num_classes=5)

In [35]:
# Model constants.
max_features = 2000
embedding_dim = 256
sequence_length = 70

In [36]:
vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [37]:
vectorize_layer.adapt(x_train)

x_train_vect = vectorize_layer(x_train)
x_val_vect = vectorize_layer(x_val)
x_test_vect = vectorize_layer(x_test)

In [38]:
len(vectorize_layer.get_vocabulary())

2000

In [39]:
sorted(vectorize_layer.get_vocabulary())[-5:]

['youreyou', 'yours', 'yourself', 'zero', 'zip']

In [40]:
np.max(x_train_vect)

1999

In [41]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")

l = layers.Embedding(max_features, embedding_dim)(inputs)
l = layers.Dropout(0.5)(l)

l = layers.Conv1D(256, 4, padding="valid", activation="relu", strides=2)(l)

l = layers.GlobalMaxPooling1D()(l)

l = layers.Dense(256, activation="relu")(l)
l = layers.Dropout(0.5)(l)

predictions = layers.Dense(5, activation="softmax", name="predictions")(l)

model = tf.keras.Model(inputs, predictions)

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [42]:
# Convert training data into TensorFlow datasets

n_batches = 128

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_vect, y_train)).batch(n_batches)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val_vect, y_val)).batch(n_batches)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.2856 - loss: 1.4919 - val_accuracy: 0.3027 - val_loss: 1.4313
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.3372 - loss: 1.4226 - val_accuracy: 0.3887 - val_loss: 1.3531
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4622 - loss: 1.2982 - val_accuracy: 0.5117 - val_loss: 1.1443
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6164 - loss: 1.0194 - val_accuracy: 0.5977 - val_loss: 1.0160
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7084 - loss: 0.7960 - val_accuracy: 0.5977 - val_loss: 1.0274
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7699 - loss: 0.6538 - val_accuracy: 0.6113 - val_loss: 1.0586
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7963 - loss: 0.5621 - val_accuracy: 0.6230 - val_loss: 1.1098
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8224 - loss: 0.5050 - val_accuracy: 0.6211 - v

In [43]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_vect, y_test)).batch(n_batches)

model.evaluate(test_dataset)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6202 - loss: 1.2281 


[1.212525486946106, 0.6256590485572815]

# End to End Model

In [44]:
# A string input
inputs = keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices

indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [45]:
# Get sentiment for any string

def get_sentiment(input_string):

    input_string = lemmatize_sentence(input_string)
    input_tensor = tf.convert_to_tensor([input_string])
    pred = end_to_end_model(input_tensor)
    emotion = labels[np.argmax(pred)]
    return emotion

In [46]:
get_sentiment("I am happy")

'hap'